In [1]:
import numpy as np
import pandas as pd
import string
from fredapi import Fred

In [2]:
# 컬럼과 행을 모두 다 출력할 수 있도록 설정
pd.set_option('display.max_columns', None)  # 열 생략 없이 보기
pd.set_option('display.max_rows', None)     # 행 생략 없이 보기
pd.set_option('display.max_colwidth', None) # 컬럼 너비 제한 해제

In [3]:
df = pd.read_csv('../../data/test/lending_club_2020_test.csv', low_memory = False)
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,3697367,20000.0,20000.0,20000.0,36 months,6.03%,608.72,A,A1,RANDALL J STOLL CPA PC,6 years,MORTGAGE,125000.0,Source Verified,Mar-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=3697367,debt_consolidation,CONSOLIDATION,475xx,IN,12.53,0.0,Oct-1997,785.0,789.0,1.0,NaN,NaN,12.0,0.0,14172.0,26.2%,42.0,f,0.00,0.00,21913.551924,21913.55,20000.00,1913.55,0.0,0.00,0.0000,Mar-2016,608.35,NaN,May-2020,724.0,720.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,291526.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54000.0,NaN,NaN,NaN,5.0,26502.0,35820.0,26.9,0.0,0.0,188.0,165.0,14.0,5.0,7.0,14.0,NaN,6.0,NaN,0.0,4.0,6.0,6.0,20.0,9.0,8.0,26.0,6.0,12.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,355384.0,37905.0,49000.0,46184.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,154429912,14950.0,14950.0,14950.0,60 months,17.74%,377.53,C,C5,NaN,3 years,MORTGAGE,12000.0,Not Verified,Jul-2019,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=154429912,home_improvement,Home improvement,604xx,IL,103.20,0.0,Aug-1986,675.0,679.0,0.0,49.0,49.0,9.0,1.0,1823.0,65.1%,11.0,f,13277.51,13277.51,3760.570000,3760.57,1672.49,2088.08,0.0,0.00,0.0000,May-2020,377.53,Jun-2020,May-2020,674.0,670.0,0.0,49.0,1.0,Joint App,122000.0,11.77,Not Verified,0.0,0.0,100724.0,0.0,7.0,1.0,5.0,9.0,98901.0,95.0,0.0,1.0,1237.0,94.0,2800.0,0.0,0.0,0.0,6.0,11192.0,977.0,65.1,0.0,0.0,42.0,394.0,21.0,9.0,0.0,21.0,49.0,NaN,49.0,1.0,2.0,2.0,2.0,3.0,7.0,2.0,4.0,2.0,9.0,0.0,0.0,0.0,1.0,90.9,0.0,1.0,0.0,106896.0,100724.0,2800.0,104096.0,4830.0,665.0,669.0,Aug-1986,0.0,2.0,10.0,21.8,1.0,10.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,163922381,19125.0,19125.0,19125.0,36 months,8.81

In [4]:
# default 변수(종속변수) 생성
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off', 'Default'])]
df['default'] = df['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default'] else 0)

# loan_status 변수 제거
df.drop(columns=['loan_status'], inplace=True)

# issue_d 변수 변환
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

In [5]:
# FRED API 연결
fred = Fred(api_key="f9ef939a9a1eeff85d31ed2c984507d9")  # 🔑 실제 API 키로 대체해야 함

# 미국 3년 만기 국채 수익률(GS3) 시계열 불러오기
series = fred.get_series('GS3')  # pandas.Series 형식으로 반환됨

In [6]:
# 2. 삭제할 변수 리스트 (84개)
drop_cols = [  # 이전에 지정한 84개
    'application_type', 'grade', 'sub_grade', 'verification_status_joint', 'hardship_loan_status', 'hardship_type', 'hardship_reason', 'hardship_status',
    'deferral_term', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date',
    'hardship_length', 'hardship_dpd', 'orig_projected_additional_accrued_interest',
    'hardship_amount', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'sec_app_revol_util', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
    'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
    'sec_app_collections_12_mths_ex_med', 'annual_inc_joint', 'dti_joint', 'mths_since_last_record',
    'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'next_pymnt_d', 'inq_fi',
    'total_cu_tl', 'emp_title', 'num_actv_bc_tl', 'hardship_flag', 'title',
    'earliest_cr_line', 'funded_amnt_inv', 'id', 'loan_amnt',
    'initial_list_status', 'int_rate', 'last_credit_pull_d',
    'last_pymnt_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code',
    'pymnt_plan', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
    'total_rec_late_fee', 'total_rec_prncp', 'url', 'zip_code', 'debt_settlement_flag',
    'desc', 'member_id', 'verified_status_joint', 'sec_app_mths_since_last_major_derog',
    'disbursement_method', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage', 'settlement_term', 'application type'
    # 추가 삭제(LC 대출 절차상 LC 대출 심사 후 적용되는 걸로 생각할 수 있음
    'last_fico_range_high', 'last_fico_range_low', 'verification_status'
]

df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

In [7]:
## 문자형 변수 전처리
## 문자형 -> 숫자형으로 변환하는 경우는 이 단계에서 결측치 처리 안함
## 문자형 -> 더미변수로 변환하는 경우는 이 단계에서 결측치 처리함

# term: 'months' 떼고 수치화
df['term'] = df['term'].str.extract(r'(\d+)').astype(int)

# emp_length: 'years' 떼고 수치화, <1 years는 0.5로
def process_emp_length(x):
    if pd.isna(x):
        return np.nan  # 명시적으로 NaN 유지
    elif '< 1' in x:
        return 0.5
    elif '10+' in x:
        return 10.0
    else:
        extracted = pd.to_numeric(pd.Series(x).str.extract(r'(\d+)')[0], errors='coerce')
        return extracted.iloc[0]

if 'emp_length' in df.columns:
    df['emp_length'] = df['emp_length'].apply(process_emp_length)

# revol_util: % 떼고 수치화
def preprocess_revol_util(x):
    if pd.isna(x):
        return np.nan  # 그대로 유지
    x = str(x).strip().replace('%', '')
    try:
        return float(x)
    except:
        return np.nan  # 변환 실패한 경우도 NaN으로 유지

if 'revol_util' in df.columns:
    df['revol_util'] = df['revol_util'].apply(preprocess_revol_util)


# grade / sub_grade 수치화
grade_map = {k: v for v, k in enumerate(['A','B','C','D','E','F','G'], start=1)}
if 'grade' in df.columns:
    df['grade_num'] = df['grade'].map(grade_map)  # map은 NaN을 그대로 유지

if 'sub_grade' in df.columns:
    sub_grades = [f"{l}{n}" for l in string.ascii_uppercase[:7] for n in range(1, 6)]
    sub_grade_map = {k: v for v, k in enumerate(sub_grades, start=1)}
    df['sub_grade_num'] = df['sub_grade'].map(sub_grade_map)


# addr_state, home_ownership, application_type, purpose, verification_status 더미 변수 처리
# one hot encoding 과정에서 dummy_na = True를 적용해 NaN 값도 하나의 카테고리로 간주했음
# 즉, 더미화한 변수들의 결측치는 여기서 처리함

dummy_cols = ['addr_state', 'home_ownership', 'purpose', 'verification_status']
df = pd.get_dummies(df, columns=[col for col in dummy_cols if col in df.columns], dtype='uint8', dummy_na=True)


In [8]:
# 결측률 < 10%인 변수들 전처리

# emp_length: 결측치 = 0 + missing label
df['emp_length_missing'] = df['emp_length'].isnull().astype(int)
df['emp_length'] = df['emp_length'].fillna(0)

# percent_bc_gt_75: 결측치 0 + missing label
df['percent_bc_gt_75_missing'] = df['percent_bc_gt_75'].isnull().astype(int)
df['percent_bc_gt_75'] = df['percent_bc_gt_75'].fillna(0)

# 0으로 채운 변수들
fillna0_cols = ['collections_12_mths_ex_med', 'tot_coll_amt', 'chargeoff_within_12_mths', 'mo_sin_old_il_acct' ]
df[fillna0_cols] = df[fillna0_cols].fillna(0)

# 중앙값으로 채운 변수들
median_replacement_cols = ['tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
                           'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
                          'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd', 'num_actv_rev_tl', 
                           'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
                           'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 
                           'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 
                           'total_bc_limit', 'total_il_high_credit_limit'] 
df[median_replacement_cols] = df[median_replacement_cols].fillna(df[median_replacement_cols].median())

In [9]:
# 결측률 계산
missing_ratio = df.isnull().mean()

# 결측률이 0보다 크고 10% 미만인 변수만 선택
low_missing_cols = missing_ratio[(missing_ratio > 0) & (missing_ratio < 0.1)].index.tolist()

print(f"결측률이 10% 미만인 변수: {low_missing_cols}")
print(f"결측률이 10% 미만인 변수들의 개수:{len(low_missing_cols)}")

결측률이 10% 미만인 변수: ['dti', 'inq_last_6mths', 'revol_util', 'last_pymnt_d']
결측률이 10% 미만인 변수들의 개수:4


In [10]:
# 결측률 < 10%인 변수들 전처리

# emp_length: 결측치 = 0 + missing label
df['emp_length_missing'] = df['emp_length'].isnull().astype(int)
df['emp_length'] = df['emp_length'].fillna(0)

# percent_bc_gt_75: 결측치 0 + missing label
df['percent_bc_gt_75_missing'] = df['percent_bc_gt_75'].isnull().astype(int)
df['percent_bc_gt_75'] = df['percent_bc_gt_75'].fillna(0)

# 0으로 채운 변수들
fillna0_cols = ['collections_12_mths_ex_med', 'tot_coll_amt', 'chargeoff_within_12_mths', 'mo_sin_old_il_acct' ]
df[fillna0_cols] = df[fillna0_cols].fillna(0)

# 중앙값으로 채운 변수들
median_replacement_cols = ['tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
                           'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
                          'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd', 'num_actv_rev_tl', 
                           'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
                           'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 
                           'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 
                           'total_bc_limit', 'total_il_high_credit_limit'] 
df[median_replacement_cols] = df[median_replacement_cols].fillna(df[median_replacement_cols].median())


In [11]:
# 결측률 > 10%인 변수들 살펴보기

# 1. 전체 숫자형 변수 목록
numeric_cols = df.select_dtypes(include='number').columns

# 2. 결측률이 10% 미만인 변수 제외
remaining_numeric_cols = list(set(numeric_cols) - set(low_missing_cols))

# 3. 더미 변수(0과 1만 있는 변수) 제외
non_dummy_numeric_cols = [
    col for col in remaining_numeric_cols
    if df[col].dropna().nunique() > 2  # 고유값 개수가 2개 이하인 변수는 제외 (0, 1 또는 상수)
]

print(f"더미 변수 제외한 숫자형 변수 목록: {non_dummy_numeric_cols}")
print(f"개수: {len(non_dummy_numeric_cols)}")

더미 변수 제외한 숫자형 변수 목록: ['acc_now_delinq', 'total_il_high_credit_limit', 'last_fico_range_high', 'open_acc_6m', 'total_bal_il', 'revol_bal', 'mths_since_recent_inq', 'open_il_12m', 'num_rev_tl_bal_gt_0', 'il_util', 'open_il_24m', 'recoveries', 'total_bc_limit', 'mo_sin_rcnt_rev_tl_op', 'num_tl_30dpd', 'emp_length', 'num_rev_accts', 'tax_liens', 'collection_recovery_fee', 'num_op_rev_tl', 'num_il_tl', 'open_rv_12m', 'num_bc_tl', 'num_actv_rev_tl', 'num_sats', 'pub_rec_bankruptcies', 'delinq_amnt', 'fico_range_high', 'pct_tl_nvr_dlq', 'total_rev_hi_lim', 'open_acc', 'mo_sin_old_il_acct', 'total_bal_ex_mort', 'funded_amnt', 'max_bal_bc', 'mo_sin_rcnt_tl', 'num_accts_ever_120_pd', 'mo_sin_old_rev_tl_op', 'num_tl_op_past_12m', 'num_tl_120dpd_2m', 'num_tl_90g_dpd_24m', 'chargeoff_within_12_mths', 'bc_open_to_buy', 'num_bc_sats', 'fico_range_low', 'tot_cur_bal', 'mths_since_recent_bc', 'collections_12_mths_ex_med', 'avg_cur_bal', 'mths_since_recent_revol_delinq', 'percent_bc_gt_75', 'acc_open_pa

In [12]:
# 결측률 10% 이상이 변수들은 일괄적으로 결측값 0 대체+missing indicator 추가
for col in non_dummy_numeric_cols:
    # 결측 여부 표시하는 변수 생성 (1: 결측, 0: 결측 아님)
    df[f'{col}_missing'] = df[col].isnull().astype(int)

    # 결측값을 0으로 대체
    df[col] = df[col].fillna(0)

/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/3904844.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing'] = df[col].isnull().astype(int)
/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/3904844.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing'] = df[col].isnull().astype(int)
/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/3904844.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [13]:
# 1) 로그 변환할 연속형 변수 목록
log_cols = [
    'annual_inc', 'dti', 'max_bal_bc', 'total_rev_hi_lim', 'avg_cur_bal',
    'bc_open_to_buy', 'delinq_amnt', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'revol_bal'
]

# 결측 → 중앙값 대체 (이미 처리된 경우에는 생략해도 됩니다)
for col in log_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

# 로그 변환
for col in log_cols:
    df[col] = np.log1p(df[col])

# 2) 바이너리로 바꿀 카운트형 변수 목록
bin_cols = [
    'delinq_2yrs', 'pub_rec', 'collections_12_mths_ex_med',
    'acc_now_delinq', 'tot_coll_amt', 'chargeoff_within_12_mths',
    'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
    'pub_rec_bankruptcies', 'tax_liens'
]

# 1 이상을 1로
for col in bin_cols:
    df[col] = df[col].fillna(0)           # NA → 0
    df[col] = (df[col] >= 1).astype(int) # 0 ⇒ 0, 1 이상 ⇒ 1

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
def get_nearest_rate(issue_date, rate_series):
    if pd.isnull(issue_date):
        return np.nan
    try:
        idx = rate_series.index.get_indexer([issue_date], method='nearest')[0]
        return rate_series.iloc[idx] / 100  # % 단위 → 소수로 변환
    except Exception as e:
        print(f"Error: {issue_date} ▶ {e}")
        return np.nan


df['risk_free_rate'] = df['issue_d'].apply(lambda x: get_nearest_rate(x, series))




/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/8667205.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['risk_free_rate'] = df['issue_d'].apply(lambda x: get_nearest_rate(x, series))


In [15]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])
df['issue_d'] = pd.to_datetime(df['issue_d'])  # 혹시 모르니 이것도

/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/4242469769.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])


In [16]:
from dateutil.relativedelta import relativedelta

def create_cash_flow_from_dates(row):
    try:
        term = int(row['term'])
        default = int(row['default'])
        issue_d = row['issue_d']  # 이미 datetime 형식
        last_pymnt_d = pd.to_datetime(row['last_pymnt_d'])  # 이건 문자열일 수도 있으니 변환
        installment = float(row['installment'])
        funded_amnt = float(row['funded_amnt'])
        recoveries = float(row['recoveries'])
        collection_fee = float(row['collection_recovery_fee'])

        # 첫 현금흐름: 대출 실행
        cash_flow = [-funded_amnt]

        if pd.isnull(issue_d) or pd.isnull(last_pymnt_d):
            return np.nan

        # 몇 회차까지 납입했는지 계산
        delta = relativedelta(last_pymnt_d, issue_d)
        last_pymnt_num = delta.years * 12 + delta.months

        for month in range(1, term + 1):
            if default == 1:
                if month <= last_pymnt_num:
                    cash_flow.append(installment)
                elif month == last_pymnt_num + 1:
                    cash_flow.append(recoveries - collection_fee)
                else:
                    cash_flow.append(0)
            else:
                cash_flow.append(installment)

        return cash_flow

    except Exception as e:
        print(f"[오류] index={row.name}, error={e}")
        return np.nan


In [17]:
df['cash_flow'] = df.apply(create_cash_flow_from_dates, axis=1)

/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/3631663944.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['cash_flow'] = df.apply(create_cash_flow_from_dates, axis=1)


In [18]:
import numpy_financial as npf

def get_irr(cash_flow):
    if not isinstance(cash_flow, list) or len(cash_flow) == 0:
        return np.nan
    irr_monthly = npf.irr(cash_flow)
    if irr_monthly is None or np.isnan(irr_monthly):
        return np.nan
    return (1 + irr_monthly) ** 12 - 1  # 연환산

df['irr'] = df['cash_flow'].apply(get_irr)
df['irr'] = df['irr'].fillna(df['risk_free_rate'])

/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/2673600630.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['irr'] = df['cash_flow'].apply(get_irr)


In [19]:
# 전처리된 파일 저장
df['inq_last_6mths_missing'] = df['inq_last_6mths'].isnull().astype(int)
df.to_csv('../../data/test/data_preprocessing.csv', index=False)

/var/folders/mc/c2xlw1v55mvcpklv1sgzmldc0000gn/T/ipykernel_2611/1714135674.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['inq_last_6mths_missing'] = df['inq_last_6mths'].isnull().astype(int)
